In [1]:
import os
from skimage import io
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import torchmetrics
import random

In [2]:
class GlauDataset(Dataset):
    def __init__(self, is_train, transform=None):
        self.is_train = is_train
        self.transform = transform
        print("AHAHA")
        
        csv_file = '/kaggle/input/claheprocessed/path_specified_label_SIMPLIFIED.csv'
        df = pd.read_csv(csv_file)
        ids = df['Eye ID'].tolist()
        labels = df['Final Label'].tolist()
        ids = [_.split('/')[-1] for _ in ids]

        pos_cases = []
        neg = []
        for id, lab in zip(ids, labels):
            if lab == 1:
                pos_cases.append((id, lab))
            elif lab == 0:
                neg.append((id, lab))
            
        # reduce the number of postitive cases
        # pos_cases = pos_cases[:int(0.1*len(pos_cases))]
        random.seed(100)
        #print(len(pos_cases))
        #print(len(neg_cases))
        neg_cases = random.sample(neg, 3000)
        num_pos_tr = int(len(pos_cases) * 0.8)
        num_neg_tr = int(len(neg_cases) * 0.8)
        self.train_data = pos_cases[:num_pos_tr] + neg_cases[:num_neg_tr]
        self.val_data = pos_cases[num_pos_tr:] + neg_cases[num_neg_tr:]

    def __len__(self):
        if self.is_train:
            return len(self.train_data)
        else:
            return len(self.val_data)

    def __getitem__(self, idx):
        if self.is_train:
            img_path, lab = self.train_data[idx]
            img = io.imread('/kaggle/input/claheprocessed/clahe_all/clahe_all/' + img_path)
            img = img.astype(np.float32)
            #img = min_max_scale(img)
            img = img.transpose(2, 0, 1)
            img = torch.from_numpy(img)
            img = self.transform(img)
        else:
            img_path, lab = self.val_data[idx]
            img = io.imread('/kaggle/input/claheprocessed/clahe_all/clahe_all/' + img_path)
            img = img.astype(np.float32)
            #img = min_max_scale(img)
            img = img.transpose(2, 0, 1)
            img = torch.from_numpy(img)
            img = self.transform(img)

        lab = torch.tensor(lab, dtype=torch.float32)
        return img, lab

class GlauDatasetBalance(Dataset):
    def __init__(self, is_train, transform=None):
        self.is_train = is_train
        self.transform = transform
        
        #csv_file = '/kaggle/input/cropped-field-of-view/path_specified_label_SIMPLIFIED.csv'
        csv_file = '/kaggle/input/claheprocessed/path_specified_label_SIMPLIFIED.csv';
        df = pd.read_csv(csv_file)
        ids = df['Eye ID'].tolist()
        labels = df['Final Label'].tolist()
        ids = [_.split('/')[-1] for _ in ids]

        pos_cases = []
        neg_cases = []
        for id, lab in zip(ids, labels):
            if lab == 1:
                pos_cases.append((id, lab))
            elif lab == 0:
                neg_cases.append((id, lab))

        num_pos_tr = int(len(pos_cases) * 0.8)
        num_neg_tr = int(len(neg_cases) * 0.8)
        self.train_data = pos_cases[:num_pos_tr]
        self.train_data_neg = neg_cases[:num_neg_tr]
        self.val_data = pos_cases[num_pos_tr:] + neg_cases[num_neg_tr:]

        # Oversample positive cases to the number of training samples
        factor = num_neg_tr // num_pos_tr
        residue = num_neg_tr % num_pos_tr
        self.train_data = self.train_data * factor + self.train_data[:residue]
        
    def __len__(self):
        if self.is_train:
            return len(self.train_data)  # number of positive samples for training
        else:
            return len(self.val_data)

    def __getitem__(self, idx):
        if self.is_train:
            img_path, lab = self.train_data[idx]
            img_path_n, lab_n = self.train_data_neg[idx]
            
            img = io.imread('/kaggle/input/claheprocessed/clahe_all/clahe_all/' + img_path)
            img = img.astype(np.float32)
            img = min_max_scale(img)
            img = img.transpose(2, 0, 1)
            img = torch.from_numpy(img)
            img = self.transform(img)
            
            img_n = io.imread('/kaggle/input/claheprocessed/clahe_all/clahe_all/' + img_path_n)
            img_n = img_n.astype(np.float32)
            img_n = min_max_scale(img_n)
            img_n = img_n.transpose(2, 0, 1)
            img_n = torch.from_numpy(img_n)
            img_n = self.transform(img_n)
            
            lab = torch.tensor(lab, dtype=torch.float32)
            lab_n = torch.tensor(lab_n, dtype=torch.float32)
            return img, img_n, lab, lab_n
            
        else:
            img_path, lab = self.val_data[idx]
            img = io.imread('/kaggle/input/claheprocessed/clahe_all/clahe_all/' + img_path)
            img = img.astype(np.float32)
            img = min_max_scale(img)
            img = img.transpose(2, 0, 1)
            img = torch.from_numpy(img)
            img = self.transform(img)
            
            lab = torch.tensor(lab, dtype=torch.float32)
            return img, lab

def min_max_scale(img):
    min_val = img.min(axis=(0, 1))
    max_val = img.max(axis=(0, 1))
    img = (img - min_val)/(max_val-min_val+1e-8)
    return img

class ResNet50(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load("pytorch/vision", "resnet50")#, weights="IMAGENET1K_V2")
        self.model.fc = nn.Linear(2048, 1)

    def forward(self, x):
        return self.model(x).squeeze()
    
class ResNet18(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = torch.hub.load("pytorch/vision", "resnet18", weights="ResNet18_Weights.IMAGENET1K_V1")
        self.model.fc = nn.Linear(512, 1)

    def forward(self, x):
        return self.model(x).squeeze()

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#device = xm.xla_device()
print(device)

# Set hyperparameters
num_epochs = 25
batch_size = 32
learning_rate = 0.00001

# Balanced sampling
is_balance = False

# Initialize transformations for data augmentation
transform = transforms.Compose([
    transforms.Resize((256, 256), antialias=True),
    transforms.RandomHorizontalFlip(),
    # transforms.RandomVerticalFlip(),
    # transforms.RandomRotation(degrees=45),
    # transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.5),
    # transforms.CenterCrop(224),
    # transforms.ToTensor(),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transform_val = transforms.Compose([
    transforms.Resize((256, 256), antialias=True),
    #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

cuda


In [4]:
# Load the ImageNet Object Localization Challenge dataset
# train_dataset = torchvision.datasets.ImageFolder(
#     root='/kaggle/input/imagenet-object-localization-challenge/ILSVRC/Data/CLS-LOC/train',
#     transform=transform
# )
if is_balance:
    train_dataset = GlauDatasetBalance(is_train=True, transform=transform)
    val_dataset = GlauDatasetBalance(is_train=False, transform=transform_val)
else:
    print("HERE")
    train_dataset = GlauDataset(is_train=True, transform=transform)
    val_dataset = GlauDataset(is_train=False, transform=transform_val)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

# Load the ResNet model
#model = ResNet50()
#model = ResNet34()
model = ResNet18()

# Parallelize training across multiple GPUs
# model = torch.nn.DataParallel(model)

# Set the model to run on the device
model = model.to(device)

HERE
AHAHA
AHAHA


Downloading: "https://github.com/pytorch/vision/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 139MB/s] 


In [5]:
# Define the loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay = 5e-4)
#scheduler = torch.optim.lr_scheduler.PolynomialLR(optimizer, total_iters=num_epochs, power=1.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, gamma = 0.9, step_size = learning_rate)

In [6]:
# Train the model...
for epoch in range(num_epochs):
    # Metric
    metric = torchmetrics.classification.BinaryAUROC()
    metric_acc = torchmetrics.classification.BinaryAccuracy()
    metric.to(device)
    metric_acc.to(device)
    log_loss = 0
    num_it = 0
    for samps in train_loader:
        if is_balance:
            inputs = torch.cat((samps[0], samps[1]), dim=0)
            labels = torch.cat((samps[2], samps[3]), dim=0)
        else:
            inputs = samps[0]
            labels = samps[1]
        
        # Move input and label tensors to the device
        inputs = inputs.to(device)
        labels = labels.to(device)

        # Zero out the optimizer
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()
        
        # Log loss
        log_loss += loss.item()
        num_it += 1

        metric.update(outputs, labels)
        metric_acc.update(outputs, labels)

    scheduler.step()
    for param_group in optimizer.param_groups:
        current_lr = param_group['lr']

    auroc = metric.compute()
    acc = metric_acc.compute()

    # Print the loss for every epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Avg loss: {log_loss/num_it:.4f}, LR: {current_lr:.6f}, AUROC: {auroc:.4f}, Acc: {acc:.4f}')


    metric = torchmetrics.classification.BinaryAUROC()
    metric_acc = torchmetrics.classification.BinaryAccuracy()
    metric.to(device)
    metric_acc.to(device)
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(inputs)
            metric.update(outputs, labels)
            metric_acc.update(outputs, labels)

    auroc = metric.compute()
    acc = metric_acc.compute()
    torch.save(model, '/kaggle/working/clahe_model'+ str(epoch) +'.pt')
    print(f'Epoch {epoch+1}/{num_epochs}, AUROC: {auroc:.4f}, Acc: {acc:.4f}')

print(f'Finished Training, Loss: {loss.item():.4f}')

Epoch 1/25, Avg loss: 57.6602, LR: 0.000010, AUROC: 0.6078, Acc: 0.4852
Epoch 1/25, AUROC: 0.5053, Acc: 0.4769
Epoch 2/25, Avg loss: 56.6223, LR: 0.000010, AUROC: 0.7185, Acc: 0.4904
Epoch 2/25, AUROC: 0.5026, Acc: 0.4817
Epoch 3/25, Avg loss: 56.0583, LR: 0.000010, AUROC: 0.7789, Acc: 0.4974
Epoch 3/25, AUROC: 0.5032, Acc: 0.4777
Epoch 4/25, Avg loss: 55.5148, LR: 0.000010, AUROC: 0.8273, Acc: 0.5016
Epoch 4/25, AUROC: 0.4989, Acc: 0.4793
Epoch 5/25, Avg loss: 54.8578, LR: 0.000010, AUROC: 0.8744, Acc: 0.5030
Epoch 5/25, AUROC: 0.4993, Acc: 0.4825
Epoch 6/25, Avg loss: 54.1274, LR: 0.000010, AUROC: 0.9152, Acc: 0.5225
Epoch 6/25, AUROC: 0.5017, Acc: 0.4777
Epoch 7/25, Avg loss: 53.3827, LR: 0.000010, AUROC: 0.9408, Acc: 0.5417
Epoch 7/25, AUROC: 0.5013, Acc: 0.4825
Epoch 8/25, Avg loss: 52.6452, LR: 0.000010, AUROC: 0.9619, Acc: 0.5708
Epoch 8/25, AUROC: 0.5013, Acc: 0.4864
Epoch 9/25, Avg loss: 51.8335, LR: 0.000010, AUROC: 0.9806, Acc: 0.6065
Epoch 9/25, AUROC: 0.5022, Acc: 0.4888
E

In [7]:
torch.save(model, '/kaggle/working/cropped_images_trained_modelResNet18.pt')